In [9]:
from pdf_gene import MoonCalc ,generate_pdf
from datetime import datetime
import requests
from tqdm import tqdm
import pandas as pd


In [3]:
def final_df(date,month, year, path="D:/code/Data_2035", dst='D:/Output'):
    
    date_obj = datetime.strptime(date, "%d-%m-%Y")    
    # Convert to YYYY-MM-DD format
    converted_date = date_obj.strftime("%Y-%m-%d")
    Moon = MoonCalc(path,converted_date,month,year +" AH",dst)
    return Moon.calculate()
final_df = final_df("30-03-2025","SHAWWAL","1446")

In [23]:
final_df

,LAG TIME(Minutes),MOON ALTITUDE(Degrees),SUN_AZIMUTH(Degrees),DAZ(Degrees),ELONGATION(Degrees),ILLUMINATION(%),CRITERION,Station,SunsetTime
date,,,,,,,,,
2025-03-30,76,14,275,0,16,1.9,A,Cherat,18:32
2025-03-30,78,14,275,1,16,1.9,A,Dir,18:32
2025-03-30,78,14,276,1,16,1.9,A,Gilgit,18:23
2025-03-30,76,14,275,0,16,1.9,A,Islamabad,18:27
2025-03-30,71,14,275,-2,16,1.9,A,Jiwani,19:09
2025-03-30,70,14,275,-2,16,1.9,A,Karachi,18:47
2025-03-30,74,14,275,0,16,1.9,A,Lahore,18:21
2025-03-30,77,14,275,1,16,1.9,A,Mansehra,18:27
2025-03-30,73,14,275,-1,16,1.9,A,Multan,18:32


In [4]:
df = final_df
# Split 'STATION(Sunset)' into 'Station' and 'SunsetTime'
df[['Station', 'SunsetTime']] = df['STATION(Sunset)'].str.extract(r'^(.*?)\s*\((.*?)\)$')

# Optional: drop the original column if no longer needed
df.drop(columns=['STATION(Sunset)'], inplace=True)

# Reorder columns if desired
df = df[['Station', 'SunsetTime', 'LAG TIME(Minutes)', 'MOON ALTITUDE(Degrees)',
         'SUN_AZIMUTH(Degrees)', 'DAZ(Degrees)', 'ELONGATION(Degrees)', 
         'ILLUMINATION(%)', 'CRITERION']]

row = df
row.columns = [
    'Station', 'SunsetTime', 'LagTime', 'MoonAltitude', 'SunAzimuth',
    'DAZ', 'Elongation', 'Illumination', 'Criterion'
]


In [16]:
visibility_mapping = {
    'A': 'Easily visible',
    'B': 'Visible under perfect conditions',
    'C': 'May need optical aid to find the crescent Moon',
    'D': 'Will need optical aid to find the crescent Moon',
    'E': 'Not visible with a telescope',
    'F': 'Not visible, below the Danjon limit'
}


In [21]:
def make_prompt(row):
    # Visibility criterion description
    visibility_description = visibility_mapping.get(row['Criterion'], 'Unknown visibility criterion')

    # Interpret illumination
    illumination = float(row['Illumination'])
    illumination_description = (
        "The illumination is high, indicating good visibility."
        if illumination > 0.8 else
        "The illumination is low, which might make the Moon harder to see."
    )

    # Interpret moon altitude
    moon_altitude = float(row['MoonAltitude'])
    altitude_description = (
        "The Moon is high above the horizon, aiding visibility."
        if moon_altitude > 8 else
        "The Moon is low on the horizon, which may limit visibility."
    )

    return (
        f"On {row.name}, at station {row['Station']}, "
        f"moon altitude {moon_altitude}°, "
        f"illumination {illumination}%, "
        f"and visibility criterion '{row['Criterion']}': {visibility_description}. "
        f"{illumination_description} {altitude_description} "
        f"Based on this data, explain whether the moon is likely to be visible and why."
    )



# ✅ Function to query LLaMA 3.2 via Ollama
def query_ollama(prompt, model="llama3.2"):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": model, "prompt": prompt, "stream": False}
    )
    return response.json()["response"].strip()

tqdm.pandas()  # This enables tqdm integration with pandas

df['llama_response'] = df.progress_apply(lambda row: query_ollama(make_prompt(row)), axis=1)
print(df[['Station', 'llama_response']].head())



100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [06:40<00:00, 30.84s/it]

              Station                                     llama_response
date                                                                    
2025-03-30     Cherat  Based on the provided data, it is highly likel...
2025-03-30        Dir  Based on the given data, it's highly likely th...
2025-03-30     Gilgit  Based on the provided data, it is highly likel...
2025-03-30  Islamabad  Based on the provided data, it appears that th...
2025-03-30     Jiwani  Based on the provided data:\n\n- Date: 2025-03...



C:\Users\Administrator\AppData\Local\Temp\ipykernel_15084\3325240022.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['llama_response'] = df.progress_apply(lambda row: query_ollama(make_prompt(row)), axis=1)


In [24]:
df["llama_response"]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    for content in df["llama_response"]:
        print(content)
        print("------------xxxxxxxx----------")

Based on the provided data, it is highly likely that the moon will be visible at station Cherat on 2025-03-30 00:00:00.

Here's why:

1. Moon altitude of 14.0°: The high altitude above the horizon indicates that the moon is well above the tree line or other obstructions, making it more likely to be visible.
2. Illumination of 1.9%: This value represents the illumination percentage, which is a measure of how much sunlight is reflected from the moon's surface towards the observer. An illumination of 1.9% is relatively high, indicating that the moon is reflecting a significant amount of sunlight and should be well-lit.
3. Visibility criterion 'A': The visibility criterion 'A' indicates that the moon can be easily seen without any issues. This suggests that the conditions are optimal for observing the moon.

Considering these factors together, it's likely that the moon will be visible at station Cherat on 2025-03-30 00:00:00 due to its high altitude and sufficient illumination.
-----------

In [ ]:
# Sample DataFrame
# df = pd.DataFrame({
#     'name': ['Alice', 'Bob'],
#     'product': ['laptop', 'smartphone'],
#     'issue': ['screen not working', 'battery draining fast']
# })

df = final_df
# Generate a response per row
def generate_response(row):
    prompt = (
        f"Customer {row['name']} reported an issue with their {row['product']}: "
        f"'{row['issue']}'. Write a helpful and professional support message."
    )
    response = requests.post(
    'http://localhost:11434/api/generate',
    json={
        "model": "llama3.2",
        "prompt": prompt,
        "stream": False
    }
    )
    return response.json()

# Apply model to DataFrame
df['llama_response'] = df.apply(generate_response, axis=1)

# Show results
print(df[['name', 'llama_response']])

In [ ]:
import pandas as pd


dfd = df

# Unpack the dictionary column
df_expanded = pd.json_normalize(dfd["llama_response"])

# Concatenate the unpacked data with the original DataFrame
df_new = pd.concat([dfd, df_expanded], axis=1).drop('llama_response', axis=1)

# Display the new DataFrame
print(df_new)

In [ ]:
df_new["response"]

In [ ]:
import pandas as pd
from llama_cpp import Llama

# Load the model (use the exact filename and path)
llm = Llama(model_path="llama-3.2-1B.Q4_K_M.gguf", n_ctx=2048)

# Sample DataFrame
df = pd.DataFrame({
    'name': ['Alice', 'Bob'],
    'product': ['laptop', 'smartphone'],
    'issue': ['screen not working', 'battery draining fast']
})

# Generate a response per row
def generate_response(row):
    prompt = (
        f"Customer {row['name']} reported an issue with their {row['product']}: "
        f"'{row['issue']}'. Write a helpful and professional support message."
    )
    output = llm(prompt, max_tokens=200)
    return output["choices"][0]["text"].strip()

# Apply model to DataFrame
df['llama_response'] = df.apply(generate_response, axis=1)

# Show results
print(df[['name', 'llama_response']])


In [ ]:
generate_pdf("30-03-2025", "SHAWWAL","1446")